# STAT 628 Project 2
* Peijin Li: Data cleaning, Model building, Model selection and improvement   
* Yanshi Luo: Data cleaning, Model building, Model selection and improvement
* Wen Huang: Data cleaning, Model building, Jupyter Notebook

## I. Introduction and Motivation

Yelp is an Internet company founded in 2004 to "help people find great local businesses" by providing a platform for users to write reviews of businesses. Our research is based on 1.5 million comments data among the user-generated Yelp reviews. 

**Movtivation:** 
   1. Find out what makes a review positive or negative based on the review and a small set of attributes
   2. Propose a prediction model to predict the ratings of reviews based on the text and attributes   

In this report, we build some supervised learning models, find out some key features that affect the review level and propose a prediction model for new review.

## II.Data Summary and Cleaning :

**Data Summary:** 
1. Training set data: 1546379 records; Test set data: 1016664 records 
2. Y variable: `stars`  
3. X variables: `text` (main information source), `name`, `date`, `city`, `longitude`, `latitude`, `categories`.

**Data Cleaning:**  
1. StopWords: Delete the most common words which contain little information. The list of stopwords is mainly from NLTK libraries, excluding words that express a person's desire, like "not","will".
1. Decontracted abbreviation: e.g. 'can't' => 'can not'
1. Negation Handling: Add `not_` tag to negation field. e.g. 'I will never go there again. It is terrible!' => 'I will never not_go not_there not_again. It is terrible!'
1. Punctuations: Remove all punctuations except for `_` for negation handling.
1. Word Stem: e.g. 'happy' => 'happi' with The Porter Stemming Algorithm
1. Addition Text Handling Process (based on models): We applied different methods to transform our data to fit different model's input requirement. For example, we do TF_IDF transformation for SVM, feature extraction for the regression model.

**Feature Engineering: opinion phrases, categories, and  specific food**

Grouping data by the stars, we plot the scaled distribution for different opinion phrases, categories, and food. For example, the following figure indicates a long tail on the distribution of the number of exclamation marks in 5 star comments.  

![](./plot/ex_mark.png)

Here is part of the list of features with **non-uniform distribution** on grouped reviews level:
* Opinion  Phrases: Number of `!`, Number of Uppercase Words, etc.
* Categories: 
    * (More likely classified to stars reviews) Delis, Polish, Vegan, French, Peruvian; 
    * (More likely classified to star reviews) Chicken Wings, Fast Food, Buffets, Tex-Mex, Burgers
* Specific Food
    * Drink: cocoa, spirits, water
    * Vegetable: celery, cauliflower, escarole
    * Oil: margarine
* Specific City and Restaurant Name (See table in `~/data/city.csv` and `~/data/name.csv`)

##  III. Model Selection:

**Selection Criteria**
We applied several supervised learning models on the tidied dataset. Our main criteria is MSE.

**Summary Table for Model Selection**

<table>
  <tr>
    <th>Models</th>
    <th>Fasttext</th>
    <th>SVM(class)</th>
    <th>SVM(regreesion)</th>
    <th>RandomForest(classification)</th>
    <th>RegressionTree(classification)</th>
    <th>LDA</th>

  </tr>
  <tr>
    <td>MSE (Cross Validation)</td>
    <td>0.59</td>
    <td>0.94</td>
    <td>0.94</td>
    <td>1.19</td>
    <td>1.19</td>
    <td>> 3 </td>
  
  </tr>
  <tr>
    <td>Additional Process</td>
    <td><font color="#FF0000">FastText Word Embedding</font></td>
    <td>TF with top 10,0000 words</td>
    <td>-</td>
    <td>TFIDF</td>
    <td>TFIDF</td>
    <td>TF</td>
  </tr>
</table>

The table shows the models we constructed and corresponding  MSE. From the table we see that the Fasttext model, combined with some features from text has the lowest MSE, thus we consider it to be our final model.

##  IV. Model Clarification:

**Summary of Final Model**

Our final model is an **aggregate regression model** combining fastText classification  results and text features. 

Firstly, we get the initial fastText prediction from processed text and categories as input. 

Secondly, we combine the significant features from the feature engineering result.

Thirdly, we proposed a polynomial regression model to get the variable interpretation.

**fastText Prediction**

fastText[1] is a text classification algorithm. It combined Continuous Bag-Of-Words (CBOW) idea N-gram model and Hierarchical Softmax algorithm. Instead of predicting the probability distribution with the representation of words vector(CBOW’s idea), fastText model takes a sequence of words as input and produces a probability distribution over the predefined classes. 

Following the guideline of Text classification part [(GitHub - facebookresearch/fastText)](https://github.com/facebookresearch/fastText), we trained our model with the processed text from the whole training set and get the initial fastText prediction result (`X0` in the next section). 

**Linear Model Result**


| Feature Name              | Explanation                                          | Estimate   | Std.Error | t-value | p-value  |
|---------------------------|------------------------------------------------------|------------|-----------|:-------:|----------|
| Intercept                 | Intercept                                            | 3.5e-01  | 5.1e-02 | 6.726   | 1.75e-11 |
| X0                        | Text Score from fastText method                      | 1.4e+00  | 8.9e-02 | 15.940  | <2e-16   |
| catscore                  | Category Score from fastText method                  | -2.4e-02 | 7.6e-03 | -3.150  | 0.001633 |
| text_length               | The length of each text                             | -2.7e-04 | 1.8e-05 | -15.771 | <2e-16   |
| num_upper_words           | Number of upper words(like NOT) | -3.0e-03 | 5.2e-04 | -5.820  | 5.89e-09 |
| num_exclamation_mark      | Number of exclamation marks       | 5.5e-02  | 10.0e-04 | 54.745  | <2e-16   |
| question_mark             | Number of question marks       | -6.2e-02 | 3.1e-03 | -19.871 | <2e-16   |
| dollar                    | Number of "$" symbols         | -2.4e-02 | 1.6e-03 | -16.241 | <2e-16   |
| precent                   | Number of percent symbols     | -1.5e-02 | 5.9e-03 | -2.546  | 0.010893 |
| facebad                   | Number of ":(" symbols         | -1.1e-01 | 1.3e-02 | -8.437  | <2e-16   |
| facegood                  | Number of ":)" symbols        | 7.3e-02  | 5.3e-03 | 13.840  | <2e-16   |
| goodcat                   | Number of good categories (`categories`)         | 3.0e-02  | 9.1e-03 | 3.339   | 0.000842 |
| badcat                    | Number of bad categories  (`categories`)         | -2.1e-02 | 4.9e-03 | -4.314  | 1.60e-05 |
| rest_length               | Total length of elements which isn't an English word | 3.9e-05  | 2.2e-05 | 1.728   | 0.084068 |
| significant_city/name | The specific city/name with non-uniform distribution (dummy variable) | ...  | ... | ...   | .... |
| I(...)              | High-order term of the above variables | ... | ... | ...  |  ...   |

where

goodcatelist = ['Delis','Polish','Vegan','French','Peruvian']

badcatelist = ['Chicken Wings','Fast Food','Buffets','Tex-Mex','Burgers']

The High-order terms of the variables are to increase the accuracy of Kaggle prediction competition. Due to space limit, we only list parts of the variable statistical result in the table. From the table, we can see all the variables are significant. The R-square of the model is **0.7356**.

**Model Diagnostics**

![](./plot/rp.png)

We randomly selected 500 samples of the prediction result and plot their residuals. We repeat 10 times and get similar plots. This implies a constant expectation around zero and homoscedasticity. Also, we assume all customers give independent comments, so Gaussian-Markov assumption holds.

##  V. Conclusion:

**Explanation of our final Model**

Our simplified model can be expressed as:

    stars ~ X0 + catscore + text_length + num_upper_words + num_exclamation_mark
               + question_mark + dollar + precent + facebad + facegood + goodcat 
               + badcat + rest_length + [city/name] + [Higher-order term]

where `X0` and `catscore` are separately text and category fastTest scores.

Other variables are some meaningful features we dug from the comments and the properties of restaurants. Our model indicates that the ranking of a restaurant can be linearly interpreted by the above features.

**Strength and Weaknesses**

* Strength
    * Our final model is a simple linear model. it's interpretable and easy to understand.   
    * The prediction result is good. (less than 0.7 RMSE on Kaggle)

* Weakness 
    * To get a high accuracy model, the quality of the variables is relatively large and results in a high collinearity, which effects the coefficient.
    * The model accuracy is not very competitive in the Kaggle contest.


### Reference

[1] fastText: https://arxiv.org/pdf/1607.01759v1.pdf